In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D,MaxPool2D,Dropout,Flatten
from keras.callbacks import History
from keras.optimizers import Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
from keras.models import load_model
from keras.initializers import glorot_uniform
import os
from ann_visualizer.visualize import ann_viz;
from matplotlib import pyplot as plt


In [ ]:
"Read the .csv file"
data = pd.read_csv('data.txt', header = None, sep='\t')

In [ ]:
min_val = data[2].min()
min_val

In [ ]:
data[2] = data[2] / abs(min_val)

In [ ]:
" Select random data for training: "
idx = np.random.choice(np.arange(len(data.values)), int(0.9 * len(data.values)), replace=False)

" Assign training dataset:  "
train_in = data.values[idx, 0:2]
train_out = data.values[idx, 2]

" Assign testing dataset: "
test_data = np.delete(data.values, idx, 0)
test_in = test_data[:, 0:2]
test_out = test_data[:, 2]

In [ ]:
train_in

In [ ]:
"Define optimizer here"
adam = Adam(lr = 0.005)
history = History()

"Define Neural Network here"
nnet = Sequential()

"Create Neural Network architecture"
"Input: Distance b/w the two robots - dim = 4"
"Output: Noise acceleration - dim = 1"
initializer = glorot_uniform()
nnet.add(Dense(4, kernel_initializer = initializer,input_dim = 2, activation = 'sigmoid'))
nnet.add(Dense(12, activation = 'sigmoid'))
nnet.add(Dense(16, activation = 'sigmoid'))
nnet.add(Dense(24, activation = 'sigmoid'))
nnet.add(Dense(12, activation = 'sigmoid'))
nnet.add(Dense(8, activation = 'sigmoid'))
nnet.add(Dense(4, activation = 'sigmoid'))
nnet.add(Dense(1, activation = 'tanh'))

"Compile the Neural Network"
nnet.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mse'])



In [8]:
"Print Model Summary"
print(nnet.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_10 (Dense)             (None, 12)                60        
_________________________________________________________________
dense_11 (Dense)             (None, 16)                208       
_________________________________________________________________
dense_12 (Dense)             (None, 24)                408       
_________________________________________________________________
dense_13 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 36        
__________

AttributeError: 'NoneType' object has no attribute 'image'

In [ ]:
"Run Curve fitting on the previously defined Neural Network"
nnet.fit(train_in, train_out, epochs=1000, batch_size=1000, callbacks=[history], verbose=1)
train_loss = history.history['loss']


In [ ]:
"Evaluate the newly created Neural Network using the testing dataset as defined earlier"
scores = nnet.evaluate(test_in, test_out)
train_loss = np.array(train_loss)

"Print the necessary Neural Network evaluation metrics"
print("\n%s: %0.2e" % (nnet.metrics_names[1], scores[1]))

In [ ]:
plt.plot(train_loss)
plt.show()
plt.plot(scores)

In [ ]:
" Try Data: "
N = 1000
dx = np.linspace(-10, 10, N).reshape([-1, 1])
dy = (np.ones(N)*1).reshape([-1, 1])
data_in = np.concatenate((dx, dy), axis = 1)

data_out = nnet.predict(data_in)
plt.plot(dx, data_out)

In [ ]:
"Save Model"
nnet.save("my_model_2.h5")


In [4]:
"Load Model"
nnet = load_model('my_model_2.h5')

In [10]:
input = tf.keras.Input(shape=(100,), dtype='int32', name='input')
x = tf.keras.layers.Embedding(
    output_dim=512, input_dim=10000, input_length=100)(input)
x = tf.keras.layers.LSTM(32)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
model = tf.keras.Model(nnet)

plot_model(nnet, show_shapes=True, show_layer_names=False, rankdir='UD')

In [ ]:
plt.plot(train_out)

In [ ]:
data_in.shape

In [ ]:
np.random.randint(4, size=(4,5))

In [ ]:
d = 5
x = np.array([[0, 0], [d, 0], [d, d], [0, d], [0, 2*d]]).T
x

In [ ]:
plt.plot(x[0,:], x[1,:],'.')

In [ ]:
def compute_dist_comp_matrix(pos_diff_y):
    num_agents = 5
    W = np.zeros((num_agents, num_agents))
    for ii in range(num_agents):
        for jj in range(num_agents):
            if pos_diff_y[ii, jj] < 0:
                W[ii, jj] = 1
            else:
                W[ii, jj] = 0
    return W

In [ ]:
def compute_pos_diff_matrix(x, y):
    num_agents = 5
    del_x = np.zeros((num_agents, num_agents))
    del_y = np.zeros((num_agents, num_agents))
    for ii in range(num_agents):
        for jj in range(num_agents):
            del_x[ii, jj] = x[jj] - x[ii]
            del_y[ii, jj] = y[jj] - y[ii]
            
    return del_x, del_y

In [ ]:
def compute_dist_comp_mat(W, dx, dy):
    num_agents = 5
    dist_comp_mat = np.zeros((num_agents, num_agents))
    for ii in range(num_agents):
        for jj in range(num_agents):
            dist = downwash(dx[ii, jj], dy[ii, jj], sigma)
            comp = nnet.predict(np.array[dx[ii, jj], dy[ii, jj]]).reshape(1,2)
            dist_comp_mat[ii, jj] = 2*W[ii, jj]*(dist - 2.935402561181967**2*comp)
    
    return dist_comp_mat

In [ ]:
def compute_tot_dist_comp(M):
    return M.sum(axis=1).reshape(-1,1)

In [ ]:
M = np.array([[1,2,3],[4,5,6]])
M

In [ ]:
M.sum(axis=1).reshape(-1,1)

In [ ]:
MM=compute_tot_dist_comp(M)

In [ ]:
MM.shape